In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

In [54]:
results_mpc = pd.read_csv('mpc_results.csv')
results_mpc['timestamp'] = pd.to_datetime(results_mpc['timestamp'])

results_nn = pd.read_csv('hybrid_rl_results.csv')
results_nn['timestamp'] = pd.to_datetime(results_nn['timestamp'])

start = pd.Timestamp(year=2018,
                     month=10,
                     day=20,
                     hour=0,
                     minute=0)
end = pd.Timestamp(year=2018,
                     month=10,
                     day=30,
                     hour=0,
                     minute=0)

results_mpc_short = results_mpc[(results_mpc['timestamp'] > start) & (results_mpc['timestamp'] < end)]
results_nn_short = results_nn[(results_nn['timestamp'] > start) & (results_nn['timestamp'] < end)]

In [62]:
results_nn

,Unnamed: 0,b_fcas_power,b_fcas_price,b_fcas_type,b_fcas_actor_power,b_fcas_actor_price,b_en_actor_power,b_en_power_gen,b_en_power_load,b_en_price,...,fcas_clearing_price,en_price,r,current_state_value,next_state_value,fcas_demand,soe,timestamp,k,delta
0,0,46.05535,41.000000,gen,3.874165,-0.314283,2.030499,0.000000,3.000000e+02,54.90,...,41.000000,54.90,-16101.012895,-160.965973,-15.090668,41.00,0.000000,2018-06-01 04:30:00,0.0,-16112.47000
1,1,66.31312,41.000000,gen,1.665301,-1.074649,13.283922,0.000000,3.000000e+02,56.19,...,41.000000,56.19,-16488.012895,-293.572388,-160.965973,41.00,295.000175,2018-06-01 05:00:00,0.0,-16479.96500
2,2,122.000725,41.000000,gen,5.804046,3.831633,44.470318,0.000000,3.000000e+02,58.73,...,41.000000,58.73,-178250.362347,-2017.214233,-293.572388,41.00,590.000349,2018-06-01 05:30:00,0.0,-178235.67000
3,3,200.0,41.670000,gen,15.012062,17.090618,93.441826,0.000000,2.439995e+02,62.98,...,41.669998,62.98,-264301.249385,-4255.774414,-2017.214233,41.67,885.000524,2018-06-01 06:00:00,0.0,-264200.38000
4,4,200.0,47.670000,gen,44.327183,32.088036,136.424350,0.000000,1.000000e+02,74.24,...,47.669998,74.24,-115042.990140,-4552.920898,-4255.774414,47.67,1029.000000,2018-06-01 06:30:00,0.0,-114830.20000
5,5,200.0,75.590000,gen,89.184220,35.859577,136.206360,-300.000000,0.000000e+00,229.57,...,70.000000,229.57,68871.030333,-2951.349121,-4552.920898,75.59,1029.000000,2018-06-01 07:00:00,0.0,69098.68000
6,6,200.0,62.140000,gen,60.783527,17.751719,67.971500,-228.999770,0.000000e+00,104.33,...,62.139999,104.33,25633.152764,-2103.271973,-2951.349121,62.14,728.999777,2018-06-01 07:30:00,0.0,25780.72000
7,7,200.0,81.000000,gen,51.652092,12.308722,42.387577,-300.000000,0.000000e+00,112.42,...,70.000000,112.42,-81844.676127,-2500.012939,-2103.271973,81.00,484.429371,2018-06-01 08:00:00,0.0,-81739.51600
8,8,200.0,81.000000,gen,33.674698,6.292669,19.292900,-84.429695,0.000000e+00,103.99,...,70.000000,103.99,8779.796870,-1910.962280,-2500.012939,81.00,184.429694,2018-06-01 08:30:00,0.0,8904.79800
9,9,200.0,78.920000,gen,28.864120,5.363100,14.835064,0.000000,3.000000e+02,94.00,...,70.000000,94.00,-28199.983493,-1809.814331,-1910.962280,78.92,100.000322,2018-06-01 09:00:00,0.0,-28104.43600


In [69]:
def compute_reward_mpc(results):
    reward = []
    for i in range(len(results)):
        r = (results['battery_bid_energy_power_gen'].iloc[i] - results['battery_bid_energy_power_load'].iloc[i]) * results['battery_bid_energy_price'].iloc[i]
        if results['battery_bid_fcas_price'].iloc[i] <= results['fcas_clearing_price'].iloc[i]:
            r += abs(results['battery_bid_fcas_price'].iloc[i] * results['battery_bid_fcas_power'].iloc[i])
        reward.append(r)
    results['reward'] = reward
    return results

def compute_reward_nn(results):
    reward = []
    for i in range(len(results)):
        r = (results['b_en_power_gen'].iloc[i] - results['b_en_power_load'].iloc[i]) * results['b_en_price'].iloc[i]
        if results['b_fcas_price'].iloc[i] <= results['fcas_clearing_price'].iloc[i]:
            r += abs(results['b_fcas_price'].iloc[i] * np.float(results['b_fcas_power'].iloc[i]))
        reward.append(r)
    results['reward'] = reward
    return results

In [70]:
results_mpc = compute_reward_mpc(results_mpc)
results_nn = compute_reward_nn(results_nn)

print(results_mpc['reward'].sum())
print(results_nn['reward'].sum())

<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>


ValueError: could not convert string to float: 'tensor(-1.0104e-07)'

In [44]:
cumulative_reward_mpc = results_mpc['r'].rolling(window=48).mean()
cumulative_reward_nn = results_nn['r'].rolling(window=48).mean()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results_mpc['timestamp'], y=cumulative_reward_mpc, name='reward MPC'), secondary_y=False)
fig.add_trace(go.Scatter(x=results_nn['timestamp'], y=cumulative_reward_nn, name='reward SAC'), secondary_y=False)


fig.update_layout(
    font=dict(family='Courier New, monospace', size=18, color='black'),
    legend_orientation="h",
    legend=go.layout.Legend(
        x=0.76,
        y=0,
        traceorder="normal",
        font=dict(
            family="Courier New, monospace",
            size=17,
        )),
    )

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Cumulative Reward")
fig.show()

In [29]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=results_nn['timestamp'], y=results_nn['k'], name='k'), secondary_y=False)

# Add figure title
fig.update_layout(
    font=dict(family='Courier New, monospace', size=18, color='black'),
    legend_orientation="h",
    legend=go.layout.Legend(
        x=0.76,
        y=0,
        traceorder="normal",
        font=dict(
            family="Courier New, monospace",
            size=17,
        )),
    )

# Set x-axis title
fig.update_xaxes(title_text="Hours of day")

# Set y-axes titles
fig.update_yaxes(title_text="Risk Factor")

fig.show()